In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Tensorflow를 이용한 RNN

In [0]:
sample = " Mulmi team, cheer-up"

# 입력: " Mulmi team, cheer-u" => 출력: "Mulmi team, cheer-up"

In [5]:
set(sample)

{' ', ',', '-', 'M', 'a', 'c', 'e', 'h', 'i', 'l', 'm', 'p', 'r', 't', 'u'}

In [6]:
list(set(sample))

['M', 'i', 'h', 'a', '-', 'm', 'c', 'r', 'e', 'u', ' ', 'p', 't', 'l', ',']

In [8]:
idx2char = list(set(sample))
char2idx = {c:i for i, c in enumerate(idx2char)}
char2idx

{' ': 10,
 ',': 14,
 '-': 4,
 'M': 0,
 'a': 3,
 'c': 6,
 'e': 8,
 'h': 2,
 'i': 1,
 'l': 13,
 'm': 5,
 'p': 11,
 'r': 7,
 't': 12,
 'u': 9}

In [0]:
dic_size = len(char2idx)
hidden_size = num_classes = dic_size

# dic_size: 원핫인코딩 크기 (rnn input size)
# hidden_size: output size (최종 출력이 아니라 cell에서 빠져나가는 출력)
# num_classes: 최종 output size (필요한 경우 softmax 사용할 수 있음)

batch_size = 1    # 데이터 1개 -> 배치 -> 업데이트
sequence_length = len(sample)-1
learning_rate = 0.1

In [12]:
sample_idx = [char2idx[c] for c in sample]
sample_idx

[10, 0, 9, 13, 5, 1, 10, 12, 8, 3, 5, 14, 10, 6, 2, 8, 8, 7, 4, 9, 11]

In [0]:
# 입력, 출력 데이터 추출

xdata = [sample_idx[:-1]]
ydata = [sample_idx[1:]]

In [0]:
tf.reset_default_graph()

x = tf.placeholder(tf.int32, [None, sequence_length])
y = tf.placeholder(tf.int32, [None, sequence_length])
x_one_hot = tf.one_hot(x, depth=num_classes)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
ini_state = cell.zero_state(batch_size, tf.float32)

outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=ini_state, dtype=tf.float32)

# fully connect  <-  정확도 높이기
xfc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(xfc, num_classes, activation_fn=None)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=weights)
loss = tf.reduce_mean(sequence_loss)

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, 2)

In [37]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(50):
    cv, _ = sess.run([loss, train], feed_dict={x:xdata, y:ydata})
    res = sess.run(prediction, feed_dict={x:xdata})
    res_str = [idx2char[c] for c in np.squeeze(res)]
    print("step: ", i, ", loss:", cv, ", prediction: ", "".join(res_str))

step:  0 , loss: 2.7005525 , prediction:  emmmmmmmmmmmmmmmmmmm
step:  1 , loss: 2.5680287 , prediction:  eeeee eeeeeeeeeeeeee
step:  2 , loss: 2.3824306 , prediction:  e  m   e    eeeee-ee
step:  3 , loss: 2.2342172 , prediction:  e  m  mee   eeerr-u-
step:  4 , loss: 1.9710312 , prediction:  uu mm meem  ceerr-up
step:  5 , loss: 1.6626753 , prediction:  Mmlmmmmeem  ceeer-up
step:  6 , loss: 1.3279722 , prediction:  Mmlmm meem, ceeer-up
step:  7 , loss: 1.017026 , prediction:  Mulmm team, cheer-up
step:  8 , loss: 0.735111 , prediction:  Mulmi team, cheer-up
step:  9 , loss: 0.5166114 , prediction:  Mulmi team, cheer-up
step:  10 , loss: 0.3537025 , prediction:  Mulmi team, cheer-up
step:  11 , loss: 0.23653248 , prediction:  Mulmi team, cheer-up
step:  12 , loss: 0.1521023 , prediction:  Mulmi team, cheer-up
step:  13 , loss: 0.09912952 , prediction:  Mulmi team, cheer-up
step:  14 , loss: 0.063162826 , prediction:  Mulmi team, cheer-up
step:  15 , loss: 0.039096512 , prediction:  Mul

# Apriori 알고리즘 구현

In [38]:
!pip install mlxtend

In [0]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

## 간단한 예시

In [0]:
dataset = [
           ["사과", "치즈", "생수"],
           ["생수", "호두", "치즈", "고등어"],
           ["수박", "사과", "생수"],
           ["생수", "호두", "치즈", "옥수수"]
]

In [42]:
te = TransactionEncoder()
te_array = te.fit(dataset).transform(dataset)
te_array

array([[False,  True,  True, False, False,  True, False],
       [ True, False,  True, False, False,  True,  True],
       [False,  True,  True,  True, False, False, False],
       [False, False,  True, False,  True,  True,  True]])

In [44]:
df = pd.DataFrame(te_array, columns=te.columns_)
df

,고등어,사과,생수,수박,옥수수,치즈,호두
0,False,True,True,False,False,True,False
1,True,False,True,False,False,True,True
2,False,True,True,True,False,False,False
3,False,False,True,False,True,True,True


In [46]:
freq_item = apriori(df, min_support=0.5, use_colnames=True)
freq_item

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(사과, 생수)"
5,0.75,"(치즈, 생수)"
6,0.50,"(생수, 호두)"
7,0.50,"(치즈, 호두)"
8,0.50,"(치즈, 생수, 호두)"


In [0]:
# 연관규칙 만들 때 사용하는 함수
from mlxtend.frequent_patterns import association_rules

In [49]:
association_rules(freq_item, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사과),(생수),0.50,1.00,0.50,1.00,1.000000,0.000,inf
1,(치즈),(생수),0.75,1.00,0.75,1.00,1.000000,0.000,inf
2,(생수),(치즈),1.00,0.75,0.75,0.75,1.000000,0.000,1.0
3,(호두),(생수),0.50,1.00,0.50,1.00,1.000000,0.000,inf
4,(호두),(치즈),0.50,0.75,0.50,1.00,1.333333,0.125,inf
5,"(치즈, 호두)",(생수),0.50,1.00,0.50,1.00,1.000000,0.000,inf
6,"(생수, 호두)",(치즈),0.50,0.75,0.50,1.00,1.333333,0.125,inf
7,(호두),"(치즈, 생수)",0.50,0.75,0.50,1.00,1.333333,0.125,inf
